In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time

c:\Users\DIEAL\.conda\envs\cudatf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
print(tf.__version__)

2.6.0


In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## 1. Preparación de los datos

In [7]:
img_size = 128
data_path = "archive (5)/img_align_celeba/img_align_celeba"

In [8]:
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = (img / 127.5) - 1  # Normalización entre [-1, 1]
    return img

In [9]:
# Obtener los paths de las imágenes y cargarlas en un dataset
image_paths = [os.path.join(data_path, filename) for filename in os.listdir(data_path)]
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(64).prefetch(buffer_size=tf.data.AUTOTUNE)

## Diseño de la GAN

In [10]:
# Definir el generador
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8 * 8 * 256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        layers.Reshape((8, 8, 256)),
        
        # Aumentar a 16x16
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        # Aumentar a 32x32
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        # Aumentar a 64x64
        layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        # Aumentar a 128x128
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')  # Produce 128x128
    ])
    return model


# Diseño del discriminador
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[128, 128, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (5,5), strides=(2,2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model



## 3. Definición de las funciones de pérdida y los optimizadores

In [11]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## 4. Entrenamiento de la GAN

In [12]:
import time

def train_step(images, generator, discriminator, batch_size=128):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


def train(dataset, generator, discriminator, epochs=50):
    print('Iniciando entrenamiento...')
    for epoch in range(epochs):
        start = time.time()
        print(f'Iniciando época {epoch + 1}')
        i = 0
        for image_batch in dataset:
            print(f'Entrenando batch {i + 1} de {len(dataset)}')
            train_step(image_batch, generator, discriminator)
            i += 1

        print(f'Tiempo para la época {epoch + 1} es {time.time() - start:.2f} segundos')
        
        # Generar imágenes después de cada época para visualizar el progreso
        seed = tf.random.normal([16, 100])
        generate_and_plot_images(generator, epoch + 1, seed)

In [13]:
# Modificar la función de entrenamiento para visualizar las imágenes generadas
def generate_and_plot_images(generator, epoch, test_input):
    predictions = generator(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        img = predictions[i].numpy()  # Convertir a NumPy
        plt.imshow((img * 127.5 + 127.5).astype(np.uint8))  # Convertir a rango [0, 255]
        plt.axis('off')

    plt.show()

# Instanciar y entrenar la GAN
generator = build_generator()
discriminator = build_discriminator()

# Iniciar el entrenamiento
train(dataset, generator, discriminator, epochs=10)


Iniciando entrenamiento...
Iniciando época 1
Entrenando batch 1 de 3166


ResourceExhaustedError: failed to allocate memory [Op:Mul]

## 5. Visualización de resultados

##  Reflexión sobre el uso de un modelo de IAGen y el laboratorio

# Task 4

### 3. ¿Qué aplicaciones potenciales ven para las GANs en la industria o en la investigación?

Las Redes Generativas Antagónicas (GANs) tienen diversas aplicaciones en la industria, la investigación y la vida cotidiana. Algunas de las aplicaciones más comunes son:

1. **Generación de imágenes y videos**: se usan para crear imágenes realistas, arte y videos en campos como la publicidad, el cine y los videojuegos.
2. **Mejora de imágenes**: se utilizan para mejorar la calidad de las imágenes, eliminar ruido y aumentar la resolución.
3. **Síntesis de datos**: generan datos sintéticos para entrenar modelos de aprendizaje automático cuando los datos reales son escasos o costosos.
4. **Estudios Biométricos**: se utilizan para generar imágenes realistas de rostros y cuerpos humanos para investigaciones en biometría y seguridad.
5. **Diseño de moda y productos**: se usan para crear diseños de moda, muebles, automóviles y otros productos.
6. **Realidad Aumentada y Virtual**: se utilizan para crear entornos virtuales y objetos 3D realistas en aplicaciones de realidad aumentada y virtual.
7. **Arte Generativo**: se utilizan para crear arte generativo y obras de arte interactivas.
8. **Simulación y Modelado**: se utilizan para simular fenómenos naturales, como el clima, la física y la biología.

### 4. ¿Qué limitaciones o preocupaciones éticas pueden identificar en el uso de GANs?

Las Redes Generativas Antagónicas (GANs) presentan varias limitaciones y preocupaciones éticas, entre las que se incluyen:

1. **Generación de Contenido Falso**: Pueden crear imágenes, videos o audios realistas pero falsos, facilitando la desinformación y la manipulación en medios sociales.

2. **Privacidad y Consentimiento**: Las GANs pueden generar datos sintéticos de personas sin su consentimiento, lo que plantea preocupaciones sobre la privacidad y el uso indebido de la información personal.

3. **Propiedad Intelectual**: La creación de contenido a partir de datos de entrenamiento puede infringir derechos de autor, ya que puede replicar el estilo o el trabajo de artistas y creadores sin reconocimiento.

4. **Sesgos en el Modelo**: Si los datos de entrenamiento contienen sesgos, las GANs pueden perpetuarlos o amplificarlos, resultando en resultados discriminatorios o inexactos.

5. **Impacto Laboral**: La automatización y generación de contenido mediante GANs pueden amenazar empleos en sectores creativos y de producción.

6. **Responsabilidad y Rendición de Cuentas**: La dificultad para rastrear la procedencia del contenido generado puede complicar la asignación de responsabilidad por el uso indebido.

